In [1]:
from tqdm.notebook import tqdm
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd.profiler as profiler

import matplotlib.pyplot as plt

import torchvision
import torchvision.transforms as transforms


LOG_PATH = "./data/profile_info/"
DATA_DIR = './data/MNIST'

In [2]:
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_VISIBLE_DEVICES=5


In [3]:
def load_data(data_dir, batch_size):
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])
    
    trainset = torchvision.datasets.MNIST(
        root=data_dir, train=True, download=True, transform=transform)

    testset = torchvision.datasets.MNIST(
        root=data_dir, train=False, download=True, transform=transform)
    
    train_loader = torch.utils.data.DataLoader(trainset, batch_size)
    test_loader = torch.utils.data.DataLoader(testset, batch_size)

    return train_loader, test_loader

In [4]:
in_channels = 3
out_channels = 10
device = "cuda:0"

train_loss = nn.CrossEntropyLoss()

learning_rate = 1e-4

batch_size = 256
epoch_num = 30

In [5]:
trainset, testset = load_data(data_dir=DATA_DIR, batch_size=batch_size)

In [6]:
def train(net, optimizer, criterion, epoch_num, train_data, device):
    """Neural network training process.
    
    """
    for epoch in tqdm(range(epoch_num)):  # loop over the dataset multiple times
        for data, target in train_data:
            inputs, labels = data.to(device), target.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()    
    return net

In [7]:
class Network(nn.Module):
  def __init__(self, channels_list):
    super().__init__()
    channels_list.insert(0, 1)
    self.convs = nn.ModuleList([
        nn.Sequential(
            nn.Conv2d(in_channels=in_ch, out_channels=out_ch, kernel_size=3),
            nn.BatchNorm2d(out_ch),
            nn.ReLU()
        )        
        for in_ch, out_ch in zip(channels_list[:-1], channels_list[1:])
    ])
    self.out = nn.Linear(in_features=channels_list[-1], out_features=10)

  def forward(self, tensor):
    for layer in self.convs:
        tensor = F.pad(tensor, (1, 1, 1, 1), "constant", 0) 
        tensor = layer(tensor)
        tensor = F.max_pool2d(tensor, kernel_size=2) 

    # output
    tensor = nn.AdaptiveAvgPool2d((1,1))(tensor) 
    tensor = torch.flatten(tensor, 1)
    tensor = self.out(tensor)
    tensor = torch.sigmoid(tensor)

    return tensor

In [8]:
net = Network([8, 16, 32])
net.to(device)

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [9]:
with torch.autograd.profiler.profile(use_cuda=True) as prof_train:
    net = train(
                net, optimizer, train_loss, epoch_num, trainset, device
    )

  0%|          | 0/30 [00:00<?, ?it/s]

MemoryError: std::bad_alloc

In [ ]:
with open(LOG_PATH + "prof_pad_outside_train.txt", "w") as file:
    file.write(prof_train.table(sort_by="self_cuda_time_total"))

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof_inference:
    net(testset)

In [ ]:
with open(LOG_PATH + "prof_pad_outside_inference.txt", "w") as file:
    file.write(prof_inference.table(sort_by="self_cuda_time_total"))

In [ ]:
# https://docs.nvidia.com/deeplearning/frameworks/pyprof-user-guide/profile.html 
# https://pytorch.org/tutorials/beginner/profiler.html